In [ ]:
# Import all libaries 

import pandas as pd 
import plotly.express as px
import matplotlib.pyplot as plt
import streamlit as st



In [ ]:
# Import Data from Google Sheets, using a loop 

gids =["1952168269","1020209628","887547605", "1168409358", "1058701335", "917968550", "701342270", "1186064842"]
spreadsheet_id = "1BV0koOEqQs580tEPGv9bpZYUfY8q8UTfZGTcEoK_VtQ"
temp = {"IMF_data_df":"",
        "urban_population_df":"",
        "income_group_df":"",
        "life_expectancy_df":"",
        "gini_index_df":"",
        "poverty_rate_df":"",
        "birth_rate_df":"",
        "health_expenditure_df":""
        }

for gid, t in zip(gids, temp):
    
    temp[t]=pd.read_csv(f"https://docs.google.com/spreadsheets/d/{spreadsheet_id}/export?format=csv&gid={gid}")



In [22]:
temp["health_expenditure_df"].head(2)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2015,2016,2017,2018,2019,2020,2021,2022,2023,2024
0,Aruba,ABW,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Africa Eastern and Southern,AFE,Current health expenditure (% of GDP),SH.XPD.CHEX.GD.ZS,NaN,NaN,NaN,NaN,NaN,NaN,...,5.971391,6.067786,5.929017,5.789155,5.812676,6.035806,6.044468,5.72792,NaN,NaN


In [23]:
# Converting IMF dataframe

# SELECTING COLUMNS TO KEEP AND TO MELT
columns_to_keep = ["COUNTRY", "INDICATOR", "UNIT"]
columns_to_melt = ["1980","1981","1982","1983","1984","1985","1986","1987", "1988", "1989", "1990","1991","1992", "1993","1994","1995","1996","1997","1998","1999","2000","2001","2002","2003","2004","2005","2006","2007","2008","2009","2010","2011","2012","2013","2014","2015","2016","2017","2018","2019","2020","2021","2022","2023","2024","2025","2026","2027","2028","2029","2030" ]

# melting
melted_IMF_df = pd.melt(temp["IMF_data_df"], id_vars=columns_to_keep, value_vars = columns_to_melt, var_name="YEAR", value_name="VALUE")
melted_IMF_df

# selecting indicators and countries

df_countries = melted_IMF_df[melted_IMF_df['COUNTRY'].isin(['Germany', 'Denmark', 'Poland','United States','Chile','Costa Rica','Japan','South Africa', 'Ghana','Ivory Coast'])] 

df_countries

# Select specific indicators: GDP per capita, Unemployment, Consumer Price Index (Inflation), Savings, Investment  

df_countries_analysis = df_countries[df_countries['INDICATOR'].isin(['Gross domestic product (GDP) per capita, current prices, U.S. dollars','Gross national savings, Percent of GDP','All Items, Consumer price index (CPI), End-of-period (EoP)','Unemployment rate'])]

df_countries_analysis


,COUNTRY,INDICATOR,UNIT,YEAR,VALUE
152,United States,"Gross national savings, Percent of GDP",Percent,1980,22.059
156,United States,Unemployment rate,NaN,1980,7.175
237,United States,"All Items, Consumer price index (CPI), End-of-...",Index,1980,86.750
273,Germany,Unemployment rate,NaN,1980,3.359
340,Germany,"All Items, Consumer price index (CPI), End-of-...",Index,1980,NaN
...,...,...,...,...,...
412147,Costa Rica,"All Items, Consumer price index (CPI), End-of-...",Index,2030,127.866
412150,Chile,"All Items, Consumer price index (CPI), End-of-...",Index,2030,168.518
412165,Chile,"Gross national savings, Percent of GDP",Percent,2030,22.257
415384,Ghana,"All Items, Consumer price index (CPI), End-of-...",Index,2030,1177.280


In [ ]:
# Renaming indicators 

indicator_map = {
'Gross domestic product (GDP) per capita, current prices, U.S. dollars': 'GDP per capita (adjusted for purchasing power parity)',
'Gross national savings, Percent of GDP': 'National savings (% GDP)',
'All Items, Consumer price index (CPI), End-of-period (EoP)': 'Inflation (CPI, %))',
'Unemployment rate': 'Unemployment levels (%)'}

df_countries_analysis['INDICATOR'] = df_countries_analysis['INDICATOR'].map(indicator_map)




/var/folders/y5/yvs2_16n2ns9rdh4zn1ffl400000gn/T/ipykernel_91087/1744296098.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_countries_analysis['INDICATOR'] = df_countries_analysis['INDICATOR'].map(indicator_map)


,COUNTRY,INDICATOR,UNIT,YEAR,VALUE
152,United States,National savings (% GDP),Percent,1980,22.059
156,United States,Unemployment levels (%),NaN,1980,7.175
237,United States,"Inflation (CPI, %))",Index,1980,86.750
273,Germany,Unemployment levels (%),NaN,1980,3.359
340,Germany,"Inflation (CPI, %))",Index,1980,NaN
...,...,...,...,...,...
412147,Costa Rica,"Inflation (CPI, %))",Index,2030,127.866
412150,Chile,"Inflation (CPI, %))",Index,2030,168.518
412165,Chile,National savings (% GDP),Percent,2030,22.257
415384,Ghana,"Inflation (CPI, %))",Index,2030,1177.280


In [ ]:
# Transform Unit of Unemployment by adding a new column

df_countries_analysis["new_col"]=df_countries_analysis[["INDICATOR", "UNIT"]].apply(lambda row: "Percent" if row["INDICATOR"]=="Unemployment levels (%)" else row["UNIT"], axis=1)
df_countries_analysis

df_countries_analysis["new_col"]

# Renaming column 

df_countries_analysis = df_countries_analysis.rename(
    columns={"new_col": "Unit"}
)

df_countries_analysis


In [54]:
# Filter for Germany 

list_countries=["United States","Germany"]

df_countries_analysis[df_countries_analysis["COUNTRY"]== "Germany"].sort_values(by=["INDICATOR", "YEAR"])


,COUNTRY,INDICATOR,YEAR,VALUE,Unit
340,Germany,"Inflation (CPI, %))",1980,NaN,Index
8548,Germany,"Inflation (CPI, %))",1981,NaN,Index
16756,Germany,"Inflation (CPI, %))",1982,NaN,Index
24964,Germany,"Inflation (CPI, %))",1983,NaN,Index
33172,Germany,"Inflation (CPI, %))",1984,NaN,Index
...,...,...,...,...,...
377841,Germany,Unemployment levels (%),2026,3.368,Percent
386049,Germany,Unemployment levels (%),2027,3.037,Percent
394257,Germany,Unemployment levels (%),2028,2.949,Percent
402465,Germany,Unemployment levels (%),2029,2.907,Percent


In [55]:
# Sort by indicator for Germany

df_countries_filered=df_countries_analysis[df_countries_analysis["COUNTRY"]=="Germany"]

df_countries_filtered = df_countries_analysis.loc[
    (df_countries_analysis["INDICATOR"] == "Unemployment levels (%)") &
    (df_countries_analysis["COUNTRY"] == "Germany")
]

df_countries_filered


,COUNTRY,INDICATOR,YEAR,VALUE,Unit
273,Germany,Unemployment levels (%),1980,3.359,Percent
340,Germany,"Inflation (CPI, %))",1980,NaN,Index
467,Germany,National savings (% GDP),1980,21.462,Percent
8481,Germany,Unemployment levels (%),1981,4.831,Percent
8548,Germany,"Inflation (CPI, %))",1981,NaN,Index
...,...,...,...,...,...
402532,Germany,"Inflation (CPI, %))",2029,143.726,Index
402659,Germany,National savings (% GDP),2029,25.566,Percent
410673,Germany,Unemployment levels (%),2030,2.912,Percent
410740,Germany,"Inflation (CPI, %))",2030,146.855,Index


In [56]:
df_countries_analysis

,COUNTRY,INDICATOR,YEAR,VALUE,Unit
152,United States,National savings (% GDP),1980,22.059,Percent
156,United States,Unemployment levels (%),1980,7.175,Percent
237,United States,"Inflation (CPI, %))",1980,86.750,Index
273,Germany,Unemployment levels (%),1980,3.359,Percent
340,Germany,"Inflation (CPI, %))",1980,NaN,Index
...,...,...,...,...,...
412147,Costa Rica,"Inflation (CPI, %))",2030,127.866,Index
412150,Chile,"Inflation (CPI, %))",2030,168.518,Index
412165,Chile,National savings (% GDP),2030,22.257,Percent
415384,Ghana,"Inflation (CPI, %))",2030,1177.280,Index
